In [9]:
"""
Robust recommender (PPMI + co-occurrence) with noise filtering.
Requirements: pandas, numpy, tqdm
pip install pandas numpy tqdm
"""

import os, re, json, math
from collections import Counter, defaultdict
from itertools import combinations
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

# ---------------- CONFIG ----------------
ORDER_CSV = "order_data.csv"
TEST_CSV = "test_data_question.csv"
OUTPUT_CSV = "recommendations_output.csv"

TOP_K = 3
VERBOSE = True

# thresholds/tunables
MIN_CO_OCCURRENCE = 1         # keep all pairs >= this
PPMI_EPS = 1e-12
GAMMA_POP = 0.3               # popularity fallback weight
ALPHA_PPMI = 1.0              # weight for ppmi*strength
# ---------------------------------------

def vprint(*a, **k):
    if VERBOSE:
        print(*a, **k)

# ---------------- parsing utilities ----------------
def normalize_display(s):
    if s is None: return None
    s2 = str(s).strip()
    if not s2: return None
    s2 = re.sub(r'\s+', ' ', s2)           # collapse whitespace
    s2 = s2.replace('™','').replace('®','')
    return s2

def lower_norm(s):
    ns = normalize_display(s)
    return ns.lower() if ns is not None else None

def try_json_extract_item_names(raw):
    """
    Try to parse JSON and find item display strings in common keys.
    Returns list or None.
    """
    try:
        obj = json.loads(raw)
    except Exception:w
        return None
    names = []
    # patterns found in dataset: orders -> item_details -> item_name
    def collect(o):
        if isinstance(o, dict):
            # check keys with lists of items
            for k,v in o.items():
                if isinstance(v, list):
                    for x in v:
                        collect(x)
                elif isinstance(v, dict):
                    collect(v)
                elif isinstance(v, str):
                    # try keys that look like item names
                    if k.lower() in ('item_name','name','title','product_name','description'):
                        names.append(v.strip())
            # also inspect keys 'orders','items','line_items'
            if 'orders' in o and isinstance(o['orders'], list):
                for it in o['orders']:
                    collect(it)
            if 'item_details' in o and isinstance(o['item_details'], list):
                for it in o['item_details']:
                    collect(it)
        elif isinstance(o, list):
            for e in o:
                collect(e)
        elif isinstance(o, str):
            names.append(o.strip())
    collect(obj)
    # dedupe and return
    out = []
    for n in names:
        if n and n not in out:
            out.append(n)
    return out if out else None

def regex_extract_item_names(s):
    # specific patterns: "item_name": "XYZ"
    if not isinstance(s, str): return None
    names = []
    for m in re.finditer(r'"item_name"\s*:\s*"([^"]+)"', s, flags=re.IGNORECASE):
        names.append(m.group(1).strip())
    for m in re.finditer(r"'item_name'\s*:\s*'([^']+)'", s, flags=re.IGNORECASE):
        names.append(m.group(1).strip())
    return names if names else None

def fallback_split(s):
    # split on common delimiters
    parts = re.split(r'\s*\|\|\s*|\s*;;\s*|\s*\|\s*|\s*;\s*|\s*\/\s*|\s*\n\s*|\s*,\s*', s)
    parts = [p.strip() for p in parts if p and p.strip()]
    return parts

def parse_order_field(field):
    """
    Return a list of display strings extracted from ORDERS column for a single row.
    """
    if field is None:
        return []
    raw = str(field).strip()
    if raw == "":
        return []
    # Try JSON structured extraction
    j = try_json_extract_item_names(raw)
    if j: return [normalize_display(x) for x in j if normalize_display(x)]
    # regex
    r = regex_extract_item_names(raw)
    if r: return [normalize_display(x) for x in r if normalize_display(x)]
    # fallback splits
    parts = fallback_split(raw)
    if len(parts) >= 2:
        return [normalize_display(x) for x in parts if normalize_display(x)]
    # final fallback: return full string
    return [normalize_display(raw)]

# ---------------- noise detection ----------------
NOISE_PATTERNS = [
    r'\bmemo\b', r'blankline', r'\bblank\b', r'\bpaid\b', r'\bpayment\b',
    r'\bnote\b', r'\binstruction\b', r'\bphone\b', r'\baddress\b', r'\bsubtotal\b',
    r'\btotal\b', r'\btip\b', r'\bcoupon\b', r'\bdiscount\b', r'\bpromo\b',
    r'\bvoid\b', r'\bcancelled\b', r'\bcanceled\b', r'\border\s*memo\b',
    r'\border\s*blank', r'order blankline'
]
NOISE_RE = re.compile('|'.join(NOISE_PATTERNS), flags=re.IGNORECASE)

def is_noise_item(display_name):
    if display_name is None: return True
    s = display_name.strip()
    if s == "": return True
    s_low = s.lower()
    # rule: explicitly match waste patterns
    if NOISE_RE.search(s_low):
        return True
    # remove entries like "Order Memo Item" etc
    if len(s) < 3:
        return True
    # if string is mostly punctuation or digits
    if re.fullmatch(r'[\W_]+', s):
        return True
    # if text is something like 'Blank', 'N/A', 'NaN'
    if s_low in {'na','n/a','nan','missing','none','null', '0 pc'}:
        return True
    return False

# ---------------- load data ----------------
if not os.path.exists(ORDER_CSV):
    raise FileNotFoundError(f"{ORDER_CSV} not found. Put it in the current folder.")

orders_df = pd.read_csv(ORDER_CSV, dtype=str, keep_default_na=False, na_values=[""])
if 'ORDERS' not in orders_df.columns:
    # try to find column
    found = None
    for c in orders_df.columns:
        if 'order' in c.lower():
            found = c; break
    if not found:
        raise ValueError("Could not find ORDERS column in order_data.csv")
    orders_col = found
else:
    orders_col = 'ORDERS'

vprint("Using ORDERS column:", orders_col)

# parse
vprint("Parsing orders (this may take a while)...")
tqdm.pandas()
orders_df['items_raw'] = orders_df[orders_col].progress_map(parse_order_field)

# display some parsed examples
vprint("Sample parsed items (first 8):")
for i, r in enumerate(orders_df['items_raw'].head(8)):
    vprint(i, r)

# normalize and build mapping (normalized key = lower-case stripped)
norm_to_display_counts = defaultdict(Counter)
def make_norm_list(raw_list):
    out = []
    for s in raw_list:
        if s is None: continue
        disp = str(s).strip()
        if disp == "": continue
        # keep display, and normalized key
        norm = disp.lower()
        out.append(norm)
        norm_to_display_counts[norm][disp] += 1
    # dedupe preserving order
    seen = set(); uniq=[]
    for x in out:
        if x not in seen:
            seen.add(x); uniq.append(x)
    return uniq

orders_df['items_norm'] = orders_df['items_raw'].map(make_norm_list)
# remove rows with no items
orders_df = orders_df[orders_df['items_norm'].map(len) > 0].reset_index(drop=True)
vprint("Orders after parsing (rows):", len(orders_df))

# ---------- filter out noisy item norms before counting ----------
vprint("Building frequency & co-occurrence (noise filtered)...")
item_freq = Counter()
co_counts = defaultdict(Counter)

for items in tqdm(orders_df['items_norm'], total=len(orders_df)):
    # filter noise and dedupe
    filtered = []
    for n in items:
        disp = norm_to_display_counts[n].most_common(1)[0][0] if n in norm_to_display_counts else n
        if not is_noise_item(disp):
            filtered.append(n)
    if len(filtered) == 0:
        continue
    unique_items = list(dict.fromkeys(filtered))
    for it in unique_items:
        item_freq[it] += 1
    for a,b in combinations(unique_items, 2):
        co_counts[a][b] += 1
        co_counts[b][a] += 1

if len(item_freq) == 0:
    raise RuntimeError("No non-noise items found after filtering. Try lowering noise rules or inspect parsed sample.")

vprint("Unique (non-noise) items:", len(item_freq))
vprint("Top 20 popular items (display -> freq):")
for it,f in item_freq.most_common(20):
    display = norm_to_display_counts[it].most_common(1)[0][0]
    vprint(f" {display} -> {f}")

# ---------- compute PPMI ----------
vprint("Computing PPMI...")
total_pairs_weight = sum(item_freq.values())  # use sum of frequencies as approx total mass
if total_pairs_weight <= 0:
    total_pairs_weight = 1.0

item_prob = {it: freq / total_pairs_weight for it, freq in item_freq.items()}
ppmi = defaultdict(dict)
for a, nbrs in co_counts.items():
    for b, co in nbrs.items():
        if co < MIN_CO_OCCURRENCE: 
            continue
        p_ab = co / total_pairs_weight
        pa = item_prob.get(a, 1e-12)
        pb = item_prob.get(b, 1e-12)
        raw = math.log((p_ab + PPMI_EPS) / (pa * pb + PPMI_EPS))
        v = max(raw, 0.0)
        if v > 0:
            ppmi[a][b] = v

# ---------- helper: display name from norm ----------
def display_name(norm):
    if norm in norm_to_display_counts and norm_to_display_counts[norm]:
        return norm_to_display_counts[norm].most_common(1)[0][0]
    return norm.title()

# ---------- detect sides/drinks keywords for business rule ----------
SIDES_KEYWORDS = ['fries','cheese','onion','corn','salad','coleslaw','rice','sandwich','tenders','tender','nugget']
DRINKS_KEYWORDS = ['soda','drink','iced tea','lemonade','pepsi','coke','sprite','soft drink','20 oz','20oz','20 oz soda']

def contains_keyword(norm, keyword_list):
    s = norm.lower()
    for kw in keyword_list:
        if kw in s:
            return True
    return False

# ---------- recommendation scorer ----------
def recommend_for_cart_display(cart_display_items, top_k=TOP_K):
    """
    cart_display_items: list of the original display strings from test file (not normalized)
    returns list of top_k display strings
    """
    # normalize test cart items to our norm keys
    cart_norms = []
    for d in cart_display_items:
        if d is None: continue
        dstr = str(d).strip()
        if dstr == "" or dstr.lower() in {'missing','na','nan'}:
            continue
        n = dstr.lower()
        if n in item_freq:
            cart_norms.append(n)
        else:
            # try to find closest normalized key by exact display match in mapping
            # sometimes test items use exact display form used in mapping
            found = None
            for norm, ctr in norm_to_display_counts.items():
                # if display equal to dstr, use that norm
                if dstr in ctr:
                    found = norm; break
            if found:
                cart_norms.append(found)
    # filter noise
    cart_norms = [c for c in cart_norms if not is_noise_item(norm_to_display_counts[c].most_common(1)[0][0])]

    # if cart empty or no known items -> fallback to top popular items
    if not cart_norms:
        out = [ display_name(it) for it,_ in item_freq.most_common(top_k) ]
        return out

    candidate_scores = defaultdict(float)
    # collect candidates from ppmi neighbors
    for c in cart_norms:
        # prefer neighbors with high PPMI weighted by co-strength
        for cand, v in ppmi.get(c, {}).items():
            if cand in cart_norms: 
                continue
            co = co_counts[c].get(cand, 0)
            # score = ppmi * log1p(co)
            candidate_scores[cand] += ALPHA_PPMI * v * math.log1p(co)

    # add popularity fallback candidates to ensure we have enough
    if len(candidate_scores) < top_k:
        for it, _ in item_freq.most_common(500):
            if it in cart_norms: continue
            candidate_scores.setdefault(it, 0.0)
            candidate_scores[it] += GAMMA_POP * item_prob.get(it, 0.0)

    if not candidate_scores:
        # final fallback
        return [ display_name(it) for it,_ in item_freq.most_common(top_k) ]

    # business rule: if cart has wings (keyword), promote sides/drinks
    promote_sides = any(('wing' in c or 'wings' in c) for c in cart_norms)
    # rank candidates
    ranked = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    ranked_norms = [it for it,_ in ranked]

    if promote_sides:
        # move items containing sides/drink keywords up
        sides = [it for it in ranked_norms if contains_keyword(it, SIDES_KEYWORDS) or contains_keyword(it, DRINKS_KEYWORDS)]
        others = [it for it in ranked_norms if it not in sides]
        ranked_norms = sides + others

    # build final, avoid duplicates and items in cart
    final = []
    for n in ranked_norms:
        if n in cart_norms: continue
        if is_noise_item(norm_to_display_counts[n].most_common(1)[0][0]): 
            continue
        if n not in final:
            final.append(n)
        if len(final) >= top_k: break

    # pad with popular if needed
    if len(final) < top_k:
        for it, _ in item_freq.most_common(500):
            if it in cart_norms or it in final: continue
            final.append(it)
            if len(final) >= top_k: break

    return [ display_name(n) for n in final[:top_k] ]

# ------------------ generate recommendations for test file ------------------
if not os.path.exists(TEST_CSV):
    raise FileNotFoundError(f"{TEST_CSV} not found.")

test_df = pd.read_csv(TEST_CSV, dtype=str, keep_default_na=False, na_values=[""])
# detect item cols
test_item_cols = [c for c in test_df.columns if re.match(r'item\d+', c.strip().lower())]
if not test_item_cols:
    # try 'item' in name
    test_item_cols = [c for c in test_df.columns if 'item' in c.lower()]
if not test_item_cols:
    raise ValueError("Couldn't detect item columns in test file. Expected item1,item2,item3 or similar.")

vprint("Detected test item columns:", test_item_cols)

output_rows = []
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    cart_display = []
    for c in test_item_cols:
        val = row.get(c, "")
        if pd.isna(val) or str(val).strip() == "": continue
        # skip tokens like 'Missing'
        if str(val).strip().lower() in {'missing','na','nan'}: continue
        cart_display.append(str(val).strip())
    recs = recommend_for_cart_display(cart_display, top_k=TOP_K)
    out = dict(row)
    out['RECOMMENDATION 1'] = recs[0] if len(recs) > 0 else ""
    out['RECOMMENDATION 2'] = recs[1] if len(recs) > 1 else ""
    out['RECOMMENDATION 3'] = recs[2] if len(recs) > 2 else ""
    output_rows.append(out)

out_df = pd.DataFrame(output_rows)
out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
vprint("Wrote", OUTPUT_CSV)

# debug prints
vprint("Sample recommendations (first 8 rows):")
print(out_df[['RECOMMENDATION 1','RECOMMENDATION 2','RECOMMENDATION 3']].head(8).to_string(index=False))


Using ORDERS column: ORDERS
Parsing orders (this may take a while)...


  0%|          | 0/1414410 [00:00<?, ?it/s]

Sample parsed items (first 8):
0 ['Order Memo Not Paid', '10 pc Grilled Wings Combo', '8 pc Grilled Wings Combo', '8 pc Spicy Wings Combo']
1 ['Ranch Dip - Regular', '50 pc Grilled Wings', 'Regular Buffalo Fries', 'Order Memo Paid']
2 ['20pc Spicy Feast Deal', 'Order Memo Paid']
3 ['Order Memo Item', 'Order Memo Paid', '20 pc Grilled Wings', 'Order Memo ASAP', 'Ranch Dip - Regular', 'Order Blankline 2', 'Order Blankline 1']
4 ['Order Blankline 2', '6 pc Grilled Wings Combo', '8 pc Grilled Wings Combo', 'Order Blankline 1', 'Order Memo ASAP', 'Order Memo Item', 'Order Memo Paid']
5 ['Order Memo Paid', '10 pc Grilled Wings Combo']
6 ['10 pc Grilled Wings', 'Order Memo Paid', 'Ranch Dip - Regular']
7 ['Order Memo Paid', '20pc Spicy Feast Deal']
Orders after parsing (rows): 1414410
Building frequency & co-occurrence (noise filtered)...


  0%|          | 0/1414410 [00:00<?, ?it/s]

Unique (non-noise) items: 138
Top 20 popular items (display -> freq):
 Ranch Dip - Regular -> 302870
 20pc Spicy Feast Deal -> 267974
 10 pc Grilled Wings Combo -> 166664
 6 pc Grilled Wings Combo -> 117894
 8 pc Grilled Wings Combo -> 117595
 Regular Buffalo Fries -> 100141
 2 pc Crispy Strips -> 84162
 Ranch Dip - Large -> 80610
 6 pc Spicy Wings Combo -> 72234
 10 pc Grilled Wings -> 67043
 Large Buffalo Fries -> 59962
 8 pc Spicy Wings Combo -> 59308
 10 pc Spicy Wings -> 59039
 Fried Corn - Regular -> 58584
 Chicken Sub Combo -> 58169
 10 pc Spicy Wings Combo -> 57397
 Flavor Platter -> 55667
 3 pc Crispy Strips Combo -> 54478
 Chicken Sub -> 45257
 15 pc Grilled Wings Combo -> 43452
Computing PPMI...
Detected test item columns: ['item1', 'item2', 'item3']


  0%|          | 0/1000 [00:00<?, ?it/s]

Wrote recommendations_output.csv
Sample recommendations (first 8 rows):
     RECOMMENDATION 1          RECOMMENDATION 2          RECOMMENDATION 3
 Cheese Dip - Regular      Fried Corn - Regular     Regular Buffalo Fries
 Cheese Dip - Regular                20 Oz Soda      Fried Corn - Regular
Regular Buffalo Fries Blue Cheese Dip - Regular    Voodoo Fries - Regular
  Large Buffalo Fries Blue Cheese Dip - Regular     Regular Buffalo Fries
  Large Buffalo Fries   Blue Cheese Dip - Large Blue Cheese Dip - Regular
   2 pc Crispy Strips         Add 5 Spicy Wings        4 pc Crispy Strips
Regular Buffalo Fries Blue Cheese Dip - Regular      Fried Corn - Regular
Regular Buffalo Fries      Fried Corn - Regular       Large Buffalo Fries
